In [1]:
import requests 
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Career Junction Scraper

1. Search for the job and area
2. Copy the link as follows:
<br> 


> ```https://www.careerjunction.co.za/jobs/results?keywords=Data+Engineer&autosuggestEndpoint=%2fautosuggest&location=0&category=&btnSubmit=+&page=```<br>
> Make sure to leave the page number out at the end



In [ ]:
def scraper(url_link):

  page = requests.get(url_link, verify=False)
  soup = BeautifulSoup(page.text, 'html.parser')

  #job titles
  job_title = []

  for job in soup.find_all('div', {'class', 'job-result-title'}):
    job_title.append(job.find('h2').find('a').text)

  #job descriptions
  job_links = []

  for job in soup.find_all('div', {'class', 'job-result-cta'}):
    job_links.append(str('https://careerjunction.co.za') + str(job.find('a', href=True).get('href')))

  job_description = []

  for link in job_links:
    link_url = str(link)
    detail_page = requests.get(link_url, verify=False)
    detail_soup = BeautifulSoup(detail_page.text, 'html.parser')

    job_description.append(detail_soup.find('div', {'class': 'job-details'}).find('p').text)

  #details on card
  salary = []
  position = []
  location = []
  updated_time = []
  expires = []
  reference = []

  for detail in soup.find_all('div', {'class', 'job-result-overview cjun-serp'}):

    salary.append((detail.find('ul').find('li', {'class', 'salary'})).text)
    position.append((detail.find('ul').find('li', {'class', 'position'})).text)
    location.append((detail.find('ul').find('li', {'class', 'location'})).text.replace("\n", ''))
    updated_time.append((detail.find('ul').find('li', {'class', 'updated-time'})).text)
    expires.append((detail.find('ul').find('li', {'class', 'expires'})).text)
    reference.append((detail.find('ul').find('li', {'class', 'cjun-job-ref'})).text)

  #combining lists into dataframe
  data = list(zip(job_title,job_description,salary, position, location, updated_time, expires, reference))
  df = pd.DataFrame(data, columns=['Job Title','Job Description', 'Salary', 'Position', 'Location', 'Updated Time', 'Expiry Date', 'Job Reference'])

  return df

def job_scraper(page_url, number_of_pages):

  url_list = []
  for i in range(1, number_of_pages+1):
    url_list.append(page_url + str(i))

  job_df = pd.DataFrame(columns=['Job Title','Job Description', 'Salary', 'Position', 'Location', 'Updated Time', 'Expiry Date', 'Job Reference'])

  for url_link in url_list:
    job_df = pd.concat([job_df, scraper(url_link)], axis=0)

  return job_df

In [ ]:
data_eng_url = 'https://www.careerjunction.co.za/jobs/results?keywords=Data+Scientist&autosuggestEndpoint=%2Fautosuggest&location=0&category=&btnSubmit=+&page='

job_scraper(data_eng_url, 7)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

KeyboardInterrupt: ignored

In [ ]:
df_career_junction_ds = job_scraper(data_eng_url, 10)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

In [ ]:
df_career_junction_ds.to_csv('career_junction_ds(02_2022).csv', index=False)

Look for:

* Experience in each descriptions / domain (ie 2+ year in Python)
* creating a tool list from a dictionary? 

In [ ]:
df_career_junction_ds

,Job Title,Job Description,Salary,Position,Location,Updated Time,Expiry Date,Job Reference
0,Data Scientist - Pricing at Datonomy Solutions,We are looking for a Data Scientist who specia...,R Undisclosed,Contract Senior position,Work From Home,Posted 13 Feb 2022,Expires in 9 days,Job 2446869 - Ref 470
1,Data Scientist (Industrial Environment),Are you a Senior Data Scientist with experienc...,Undisclosed,Contract Senior position,"Johannesburg, Gauteng",Posted 13 Feb 2022,Expires in 6 days,Job 2445178 - Ref Data Scientist
2,Data Scientist at Datonomy Solutions,Datonomy Solutions is founded on a business mo...,"R25,000 - R30,000 per month",Contract Junior position,"Cape Town, Western Cape",Posted 13 Feb 2022,Expires in 18 days,Job 2448638 - Ref 475
3,Data Scientist,Are you a Senior Data Scientist with experienc...,Undisclosed,Contract Senior position,"Johannesburg, Gauteng",Posted 13 Feb 2022,Expires in 6 days,Job 2445179 - Ref zensar contract
4,Data Scientist (with Cloud Azure essential) - ...,We are looking for a Data Science Developer wi...,R Undisclosed,Contract Senior position,"Gauteng, South Africa",Posted 13 Feb 2022,Expires in 11 days,Job 2446914 - Ref EVDB_4961667842
...,...,...,...,...,...,...,...,...
20,Domain Architect (Analytics) at Datafin Recrui...,ENVIRONMENT:A leading Retail Giant seeks the e...,R Undisclosed,Permanent Executive position,"Brackenfell, Western Cape",Posted 08 Feb 2022,Expires in 22 days,Job 2448990 - Ref 21169
21,Domain Architect – Analytics,"Position Purpose:Become part of an innovative,...",R Undisclosed,Permanent Senior position,"Cape Town Region, Western Cape",Posted 07 Feb 2022,Expires in 23 days,Job 2448814 - Ref PE009383LM
22,Patient Journey Partner Hematology and Oncology,Patient Journey Partner (Oncology and Hematolo...,"R76,600 - R149,600 per month",Permanent Management position,"Midrand, Gauteng",Posted 08 Feb 2022,Expires in 10 days,Job 2446171 - Ref DOR22/1/22
23,Patient Journey Partner Hemophilia South Africa,Patient Journey Partner (Hemophilia) based any...,"R50,800 - R116,700 per month",Permanent Management position,"Midrand, Gauteng",Posted 08 Feb 2022,Expires in 10 days,Job 2446168 - Ref DOR22/1/22


In [2]:
URL = 'https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&vjk=f4db89be39bc463d'

page = requests.get(URL)
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
for tag in soup.select(".resultContent span[title]"):
    print(tag.text)

Data Scientist
Data Scientist III
Principal Data Scientist
Data Scientist
Data Scientist
Engineer - Civil
Scientist II
Senior Data Scientist
Data Scientist
Data Scientist
3 X Scientists
Risk Advisory – Data Analytics – Evolve Analytics Academy 20...
Scientist (Production Level): Entomology, Ref No. AGR 10/202...
Jnr Data Scientist
Data Scientist


In [4]:
job_links = []

table = soup.find(id="resultsCol")
for link in table.find_all('a', attrs = {'class' : 'tapItem'}):
    job_links.append('https://www.indeed.com'+ str(link.attrs['href']))


job_links

['https://www.indeed.com/rc/clk?jk=73517d0ab1de27b9&fccid=1060127bb26d1435&vjs=3',
 'https://www.indeed.com/rc/clk?jk=5c3bec1e5d252a9a&fccid=c08740860a28e995&vjs=3',
 'https://www.indeed.com/rc/clk?jk=c3439d30980065ca&fccid=374d720d3973ca1c&vjs=3',
 'https://www.indeed.com/rc/clk?jk=e2504488c14e9fdd&fccid=714e147560da822f&vjs=3',
 'https://www.indeed.com/rc/clk?jk=71c6539c26e69322&fccid=e5b0c7e8f0872636&vjs=3',
 'https://www.indeed.com/rc/clk?jk=c94ad165927c1f7e&fccid=bafa4b35e9cd20ce&vjs=3',
 'https://www.indeed.com/rc/clk?jk=ed1dee521a7fe798&fccid=bca1c8ed48c3b338&vjs=3',
 'https://www.indeed.com/rc/clk?jk=107cc52bae4cb37d&fccid=374d720d3973ca1c&vjs=3',
 'https://www.indeed.com/rc/clk?jk=5d1e14f7d61a69e8&fccid=2a978f78453aef31&vjs=3',
 'https://www.indeed.com/rc/clk?jk=1eca2c1ff65f1668&fccid=df091578c26599b6&vjs=3',
 'https://www.indeed.com/company/Mintek/jobs/X-Scientist-a786dd6e14177850?fccid=8d30a64de239f3ca&vjs=3',
 'https://www.indeed.com/rc/clk?jk=9aead22fd564285f&fccid=9e215d8

In [5]:
job_descriptions = []

for link in job_links:
  link_url = str(link)
  detail_page = requests.get(link_url)
  detail_soup = BeautifulSoup(detail_page.text, 'html.parser')

  job_descriptions.append(detail_soup.find('div', {'class': 'jobsearch-jobDescriptionText'}).text)

In [6]:
job_descriptions

['Connecting with a multitude of stakeholders to understand the data in a healthcare and fraud context.\n\nMining large structured and unstructured datasets for data exploration to find new insights to detect healthcare fraud.\n\nResearch and application of the most up to date machine learning algorithms and AI techniques.\n\nPresent data and model findings in a way that provides actionable insights.\n\nImplementing results for use in both the internal and external environments.\n\nMonitoring model performance.\n\nIm prove processes and databases where opportunities arise\n\nMatric with either of the below qualification is essential.\nHonours degree in either Data Science, Statistics, or Applied Mathematics with some experience in software engineering, computer science or working with big disparate sets of data, OR\nHonours or Master’s degree in Computer Science or Software Engineering with solid experience in statistical modelling, data mining and machine learning.\nOther analytical q

In [7]:
def indeed_scraper(url_link):

  page = requests.get(url_link)
  soup = BeautifulSoup(page.text, 'html.parser')


  #job titles

  job_titles = []

  for tag in soup.select(".resultContent span[title]"):
    job_titles.append(tag.text)

  #job descriptions

  job_links = []

  table = soup.find(id="resultsCol")
  for link in table.find_all('a', attrs = {'class' : 'tapItem'}):
    job_links.append('https://www.indeed.com'+ str(link.attrs['href']))


  job_descriptions = []

  for link in job_links:
    link_url = str(link)
    detail_page = requests.get(link_url)
    detail_soup = BeautifulSoup(detail_page.text, 'html.parser')

    job_descriptions.append(detail_soup.find('div', {'class': 'jobsearch-jobDescriptionText'}).text)

  data = list(zip(job_titles,job_descriptions))
  df = pd.DataFrame(data, columns=['Job Title','Job Description'])

  return df

In [8]:
url_link = 'https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&vjk=f4db89be39bc463d'

indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&vjk=f4db89be39bc463d')

,Job Title,Job Description
0,Data Scientist,Connecting with a multitude of stakeholders to...
1,Data Scientist III,"About us, purpose, experience and qualificatio..."
2,Principal Data Scientist,Role Purpose\nWe employ individuals who are as...
3,Data Scientist,JOB OVERVIEW\nCareer Area: Engineering\nLocati...
4,Data Scientist,Job ID\n\n114723\n\nTalent Acquisition Consult...
5,Engineer - Civil,AECOM South Africa are looking for a senior br...
6,Scientist II,The primary responsibility of this position is...
7,Senior Data Scientist,Role Purpose\nCo-creating Advanced Machine Lea...
8,Data Scientist,Job ID: 10176\nPosition Type: Full-Time\nJob F...
9,Data Scientist,Put your skills to the test in 2022!!!\nA new ...


In [ ]:
df_1 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&vjk=ec5c34b672588286')
df_2 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=10&vjk=71c6539c26e69322')
df_3 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=20&vjk=69529e95233818b2')
df_4 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=30&vjk=1eca2c1ff65f1668')
df_5 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=40&vjk=e24b9747d699404e')
df_6 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=40&vjk=e24b9747d699404e')
df_7 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=60&vjk=317fd43451e2d193')
df_8 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=70&vjk=a181b4afc7c40873')
df_9 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=80&vjk=0da7b3353805cea1')
df_10 = indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=90&vjk=48160db5073d4d03')

In [ ]:
df_jobs = pd.concat([df_1,df_2, df_3, df_4, df_5, df_6, df_7, df_8])
df_jobs.drop_duplicates(keep=False, inplace=True)
df_jobs.to_csv('indeed_jobs(02_2022).csv', index=False)

In [11]:
indeed_scraper('https://za.indeed.com/jobs?q=data%20scientist&l=South%20Africa&start=10&vjk=71c6539c26e69322')

,Job Title,Job Description
0,Actuary (Data Scientist),Business Unit: Discovery Life\nFunction: Actua...
1,Data Scientist,Introduction\nAs a Data Scientist working at t...
2,Data Scientist,"One of SA''s fastest growing, Award Winning Fi..."
3,Laboratory Analytical Scientist,Overview\nLeads scientific investigations in Q...
4,Data Scientist,PURPOSE STATEMENT:\n\nThe Senior Data Scientis...
5,Senior Data Scientist,One of our clients a Tier 1 International Cons...
6,Data Scientist,"Duties:\nWe are dotsure.co.za, based in beauti..."
7,Graduate Civil Engineer - Geometric,Role & Responsibilities:\n+ You will be workin...
8,Mid-Level Data Scientist,RS Recruitment Services is assisting with the ...
9,REPL Group - Senior Data Scientist,We’re expanding our offering for an in-house S...
